In [3]:
import locationtagger
import nltk
from pymongo import MongoClient
from getpass import getpass
from time import sleep
from datetime import datetime, timedelta
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium import webdriver
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import os
# from collections import Counter
from dotenv import load_dotenv
load_dotenv()

from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

import certifi; 


# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_pe

True

In [4]:

def getTweetData(tweet):
    try:
        tweetTime = tweet.find_element(
            By.XPATH, ".//time").get_attribute('datetime')
    except NoSuchElementException:
        return
    try:
        # This only gets one image, what if there are more?
        tweetImage = tweet.find_element(
            By.XPATH, ".//div[1]/div[1]//div[2]/div[2]//img").get_attribute("src")
    except NoSuchElementException:
        # print("no image")
        tweetImage = "No Image"
    tweetText = tweet.find_element(
        By.XPATH, ".//div[1]/div[1]/div[2]/div[2]/div[2]").text

    tweetInfo = [tweetText, tweetTime, tweetImage]
    return tweetInfo


def ScraperMain():
    twitterUsername = os.getenv('TW_USERNAME')
    twitterPassword = os.getenv('TW_PASSWORD')
    

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

    # driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get('https://www.twitter.com/login')
    sleep(3)
    # Finding and inputing username
    username = driver.find_element(
        By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[5]/label[1]/div[1]/div[2]/div[1]/input[1]")
    username.send_keys(twitterUsername)
    username.send_keys(Keys.RETURN)
    sleep(1)
    # Finding and inputing Password
    # mypassword = getpass()
    password = driver.find_element(
        By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/label[1]/div[1]/div[2]/div[1]/input[1]")
    password.send_keys(twitterPassword)
    password.send_keys(Keys.RETURN)
    sleep(5)

    # Selecting and searching required police section
    search = driver.find_element(
        By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[2]/main[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/form[1]/div[1]/div[1]/div[1]/div[1]/label[1]/div[2]/div[1]/input[1]")
    search.send_keys('@TPSOperations')
    search.send_keys(Keys.RETURN)
    sleep(2)

    # Click required page
    SearchQuery = driver.find_element(
        By.XPATH, "//span[contains(text(),'Toronto Police Operations')]")
    SearchQuery.click()

    # Storing Tweet data in list
    tweetData = []
    tweetIds = set()
    lastPos = driver.execute_script("return window.pageYOffset;")
    scrolling = True
    maxDate = False
    stopDate = str((datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d'))

    while (scrolling and not (maxDate)):
        # Find Tweets
        TPOPageTweets = driver.find_elements(
            By.XPATH, '//article[@data-testid="tweet"]')
        sleep(1)
        for tweet in TPOPageTweets[-100:]:
            currentTweetInfo = getTweetData(tweet)
            if currentTweetInfo:
                tweetId = "".join(currentTweetInfo)
                if tweet not in tweetIds and currentTweetInfo[0] != '' and ":" in currentTweetInfo[0] and 'Good night' not in currentTweetInfo[0] and 'Good afternoon' not in currentTweetInfo[0] and 'Good morning' not in currentTweetInfo[0] and 'Good Night' not in currentTweetInfo[0]:
                    tweetIds.add(tweet)
                    currentTweetInfo.append(tweetId)
                    tweetData.append(currentTweetInfo)
                    if stopDate in currentTweetInfo[1]:
                        maxDate = True
                        break

        scrollAttempt = 0
        while True:
            driver.execute_script('window.scrollBy(0,3000);')
            sleep(2)
            currPos = driver.execute_script("return window.pageYOffset;")
            if maxDate:
                break
            if lastPos == currPos:
                scrollAttempt += 1
                if scrollAttempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(2)
            else:
                lastPos = currPos
                break
    print("Tweets Scraped")
    return tweetData

In [9]:
def sortData(scrapedTweet):
    # List: catagory (Missing...), Status(Update, Located)
    computedData = []
    counter = 0
    for i in range(len(scrapedTweet)):
        dataObject = {}
        if (scrapedTweet[i][0].find('\n') != -1):
            # Getting Status:
            dataObject["Status"] = scrapedTweet[i][0].splitlines()[0].split(":")[0]
            # Getting Updates (?LOCATED)...
            try:
                dataObject["Updates"] = scrapedTweet[i][0].splitlines()[0].split(":")[1]
            except IndexError:
                dataObject["Updates"] = ""
            # currentLocation could be the currentIdentity
            currentLocation = scrapedTweet[i][0].splitlines()[1].split(", ")
            # Checking whether we have a Location or an identity
            if any(char.isdigit() for char in currentLocation):
                dataObject["Name"] = currentLocation[0]
                if currentLocation[1].isnumeric():
                    dataObject["Age"] = currentLocation[1]
                else:
                    dataObject["Age"] = ""
                description = scrapedTweet[i][0].splitlines()
                del description[0:2]
                #Extracting Location from Description
                lists = scrapedTweet[i][0].splitlines()
                del lists[0:2]
                try:
                    partOfDescription = lists[0] + " " + lists[1]
                except IndexError:
                    partOfDescription = scrapedTweet[i][0]                   
                dataObject["Location"] = locationtagger.find_locations(text = partOfDescription).other
            else:
                try:
                    dataObject["Location"] = scrapedTweet[i][0].splitlines()[1]
                except IndexError:
                    dataObject["Location"] = ""
                description = scrapedTweet[i][0].splitlines()
                del description[0:1]
            if dataObject["Location"] is not None:
                locationcheck = dataObject["Location"]
                if type(dataObject["Location"]) == list and bool(dataObject["Location"]):
                    res = max(locationcheck, key = len)
                    locationcheck = res
                if "and" in locationcheck:
                    locationcheck = locationcheck.replace("and", "")
                if "&" in locationcheck:
                    locationcheck = locationcheck.replace("&", "")
                geolocator = Nominatim(user_agent="Twitter_Scraper")
                geocode = lambda query: geolocator.geocode("%s, Toronto ON" % query)
                geocode2 = RateLimiter(geocode, min_delay_seconds=0.001)
                sleep(1)
                locationcor = geocode2(locationcheck)
                if locationcor is not None:
                    locationcor2 = [locationcor.latitude, locationcor.longitude]
                    dataObject["LocationGoeCode"] = locationcor2
                else:
                    dataObject["LocationGoeCode"] = []
                    
            else:
                dataObject["LocationGoeCode"] = ["No Location Provided"]
            input_datetime = datetime.strptime(scrapedTweet[i][1], "%Y-%m-%dT%H:%M:%S.%fZ")
            # Format as MM-DD-YYYY
            output_date_string = input_datetime.strftime("%m-%d-%Y")
            dataObject["TweetedTime"] = output_date_string
            dataObject["ImageUrl"] = scrapedTweet[i][2]
            dataObject["Description"] = description
            computedData.append(dataObject)
            counter = counter + 1
            print(counter)
            print(dataObject['Location'])
            print(dataObject['LocationGoeCode'])
        else:
            print("Tweet Skipped: ", scrapedTweet[i])
    print("Tweets Sorted")
    return(computedData)
        


In [6]:

def addToDB(computedData):
    MONGODB_PASS = os.getenv('MONGODB_PASS')

    CONNECTION_STRING = "mongodb+srv://OCMB1:" + MONGODB_PASS + \
        "@cluster0.e5wloiz.mongodb.net/?retryWrites=true&w=majority"

    client = MongoClient(CONNECTION_STRING, tlsCAFile=certifi.where())

    if 'Toronto_Police_Crime_Report' in client.list_database_names():
        client.drop_database('Toronto_Police_Crime_Report')

    db = client['Toronto_Police_Crime_Report']
    collection_name = db["tweetsData"]
    res = collection_name.insert_many(computedData)
    print("Added To DataBase")

In [7]:
tweets = ScraperMain()
# sortedData = sortData(tweets)
# addToDB(sortedData)

Tweets Scraped


In [8]:
len(tweets)
# tweets

414

In [10]:
sortedData = sortData(tweets)


1
['Gerrard St E', 'grey goatee', 'Jan', 'Broadview Ave', 'Gerrard St']
[43.6764291, -79.3589359]
2
['the College St & Margueretta St', 'Jan', 'College St', 'Margueretta St']
[]
3
Bayview Ave & Sheppard Ave E
[43.7667011, -79.3873935]
4
Weston Road/Lawrence Avenue West area (8:33am)
[]
5
- injuries are non-life threatening
[]
6
['the Jane St & Thretheway Dr', 'chin length', 'Jan', 'Jane St', 'Thretheway Dr']
[]
7
Vaughan Rd & St Clair Ave W
[43.6827985, -79.4205888]
8
Vaughan Rd & St Clair Ave W
[43.6827985, -79.4205888]


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Lawrence Ave E  Ralph St',), **{}).
Traceback (most recent call last):
  File "c:\Users\user\Documents\Projects\Ontario_Crime_Mapper\Twitter_Scraper\venv\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\Projects\Ontario_Crime_Mapper\Twitter_Scraper\venv\Lib\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 1378, in getresponse
    response.begin()
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\

9
Lawrence Ave E & Ralph St
[]
10
['August', 'Finch Ave', 'HWY']
[43.779756, -79.4156124]
11
Eastern Ave & Trinity St
[]
12
['the Yonge St & Steeles', 'Jan', 'Yonge St', 'Steeles Ave']
[]
13
[]
[43.6534817, -79.3839347]
14
[]
[43.6534817, -79.3839347]
15
Milner/Morningside
[43.8004672, -79.1993297]
16
[]
[43.6534817, -79.3839347]
17
[]
[43.6534817, -79.3839347]
18
Milner/Morningside
[43.8004672, -79.1993297]


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Wilson Av + Allen Rd area',), **{}).
Traceback (most recent call last):
  File "c:\Users\user\Documents\Projects\Ontario_Crime_Mapper\Twitter_Scraper\venv\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\Projects\Ontario_Crime_Mapper\Twitter_Scraper\venv\Lib\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 1378, in getresponse
    response.begin()
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib

19
Wilson Av + Allen Rd area
[]


...

394
7: 11 p.m.
[]


RateLimiter caught an error, retrying (0/2 tries). Called with (*([],), **{}).
Traceback (most recent call last):
  File "c:\Users\user\Documents\Projects\Ontario_Crime_Mapper\Twitter_Scraper\venv\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\Projects\Ontario_Crime_Mapper\Twitter_Scraper\venv\Lib\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 1378, in getresponse
    response.begin()
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 27

395
[]
[43.6534817, -79.3839347]
396
COLLISION:
[43.6544629, -79.3682205]
397
Wynford Dr and Wynford Heights Cres
[]
398
Wynford Dr and Wynford Heights Cres
[]
399
[]
[43.6534817, -79.3839347]


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Victoria Park Ave  Dawes Rd',), **{}).
Traceback (most recent call last):
  File "c:\Users\user\Documents\Projects\Ontario_Crime_Mapper\Twitter_Scraper\venv\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\Documents\Projects\Ontario_Crime_Mapper\Twitter_Scraper\venv\Lib\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 1378, in getresponse
    response.begin()
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\L

400
Victoria Park Ave and Dawes Rd
[]
401
Victoria Park Ave and Dawes Rd
[]
402
[]
[43.6534817, -79.3839347]
403
Victoria Park Ave and Dawes Rd
[]
404
Victoria Park Ave and Dawes Rd
[]
405
E/B FGX  and Spadina Ave
[]
406
E/B FGX  and Spadina Ave
[]
407
[]
[43.6534817, -79.3839347]
408
[]
[43.6534817, -79.3839347]
409
[]
[43.6534817, -79.3839347]
410
[]
[43.6534817, -79.3839347]
411
[]
[43.6534817, -79.3839347]
412
Shuter St and Parliament St
[43.6570337, -79.3651482]
413
[]
[43.6534817, -79.3839347]
414
Yonge/Dundas
[43.65602765, -79.38012544759022]
Tweets Sorted


In [11]:
addToDB(sortedData)

Added To DataBase


In [ ]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Twitter")
geocode = lambda query: geolocator.geocode("%s, Toronto ON" % query)
location = geocode("Victoria Park Van Horne")
location2 = [location.latitude, location.longitude]
location2
# print((location.latitude, location.longitude))

In [ ]:
import requests

# Replace YOUR_API_KEY with your actual API key. Sign up and get an API key on https://www.geoapify.com/ 
API_KEY = "e904244da7514e13a457430791adcf16"

# Define the address to geocode
address = "Jane St Wilson Ave area, Toronto, CA"

# Build the API URL
url = f"https://api.geoapify.com/v1/geocode/search?text={address}&limit=1&apiKey={API_KEY}"

# Send the API request and get the response
response = requests.get(url)

# Check the response status code
if response.status_code == 200:
    # Parse the JSON data from the response
    data = response.json()

    # Extract the first result from the data
    result = data["features"][0]

    # Extract the latitude and longitude of the result
    latitude = result["geometry"]["coordinates"][1]
    longitude = result["geometry"]["coordinates"][0]

    print(f"Latitude: {latitude}, Longitude: {longitude}")
else:
    print(f"Request failed with status code {response.status_code}")